In [1]:
import sys
sys.path.append('/home/abk171/ultralytics')
from ultralytics import YOLO
import SimpleITK as sitk
from PIL import Image
import random
import os
import pydicom
import numpy as np
from tqdm import tqdm
import wandb

In [2]:
restructured_root = 'data_yolo'
image_dir = os.path.join(restructured_root, 'images')
image_train_dir = os.path.join(image_dir, 'train')
image_val_dir = os.path.join(image_dir, 'val')
image_test_dir = os.path.join(image_dir, 'test')
label_dir = os.path.join(restructured_root, 'labels')
label_train_dir = os.path.join(label_dir, 'train')
label_val_dir = os.path.join(label_dir, 'val')

## make these directories
os.makedirs(restructured_root, exist_ok=True)
os.makedirs(image_dir, exist_ok=True)
os.makedirs(image_train_dir, exist_ok=True)
os.makedirs(image_val_dir, exist_ok=True)
os.makedirs(image_test_dir, exist_ok=True)
os.makedirs(label_dir, exist_ok=True)
os.makedirs(label_train_dir, exist_ok=True)
os.makedirs(label_val_dir, exist_ok=True)

In [3]:
anno_dir = '/data_vault/hexai02/CarpalTunnel/Annotations'
dicom_dir = '/data_vault/hexai02/CarpalTunnel/Images'

In [4]:
from Utilities.utilities import load_mask, get_bounding_box, load_dicom, dicom_to_jpg, anno_to_yolo

In [12]:
dicom_paths = sorted(os.listdir(dicom_dir))

badset = {'97.dcm', '95.dcm', '178.dcm'}
for i in tqdm(range(len(dicom_paths)), desc="Processing DICOMs"):
    if dicom_paths[i] in badset:
        continue
    anno_path = dicom_paths[i].split('.')[0] + '.nii.gz'
    dicom_path = os.path.join(dicom_dir, dicom_paths[i])
    anno_path = os.path.join(anno_dir, anno_path)
    
    img_array = load_dicom(dicom_path)
    img_jpg = dicom_to_jpg(img_array)
    # height, width, dim = img_array.shape
    height, width = img_array.shape
    bbox = anno_to_yolo(anno_path, height, width)


    img_filename = dicom_paths[i].replace('.dcm', '.jpg')
    label_filename = img_filename.replace('.jpg', '.txt')
    
    if random.random() < 0.8:
        img_jpg.save(os.path.join(image_train_dir, img_filename))
        with open(os.path.join(label_train_dir, label_filename), 'w') as f:
            f.write(f"0 {' '.join(map(str, bbox))}\n")

    else:
        img_jpg.save(os.path.join(image_val_dir, img_filename))
        with open(os.path.join(label_val_dir, label_filename), 'w') as f:
            f.write(f"0 {' '.join(map(str, bbox))}\n")

    

Processing DICOMs: 100%|████████████████████████████████████████████| 123/123 [00:09<00:00, 13.03it/s]


In [8]:
dicom_paths[i]

'96.dcm'

In [3]:
%%writefile data_yolo.yaml
path: /home/abk171/hexai_work/data_yolo
train: images/train
val: images/val
test: images/test

names:
    0: ulnar

Overwriting data_yolo.yaml


In [5]:
from ultralytics import YOLO
from ultralytics import settings
settings.update({"wandb": True})
model = YOLO('yolo11n.pt')

In [6]:
import wandb
wandb.login(key="6e6412a1eff673cacdcda973f6e61422daaa9387")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/abk171/.netrc
wandb: Currently logged in as: abk171 (abk171-university-of-pittsburgh) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:


results = model.train(
    data='data_yolo.yaml',
    imgsz=[450, 1100],
    epochs=40,
    patience= 0,
    batch=16,
    workers=4,
    optimizer='AdamW',
    lr0=0.001, fliplr=0.5,crop_fraction=0.2,
    project="yolo",name="yolov11 40 epochs"
    )

New https://pypi.org/project/ultralytics/8.3.95 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.94 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data_yolo.yaml, epochs=40, time=None, patience=0, batch=16, imgsz=[450, 1100], save=True, save_period=-1, cache=False, device=None, workers=4, project=yolo, name=yolov11 40 epochs2, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=Fals

Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...
AMP: checks passed ✅
WARNING ⚠️ updating to 'imgsz=1100'. 'train' and 'val' imgsz must be an integer, while 'predict' and 'export' imgsz may be a [h, w] list or an integer, i.e. 'yolo export imgsz=640,480' or 'yolo export imgsz=640'
WARNING ⚠️ imgsz=[1100] must be multiple of max stride 32, updating to [1120]


train: Scanning /home/abk171/hexai_work/data_yolo/labels/t
val: Scanning /home/abk171/hexai_work/data_yolo/labels/val


Plotting labels to yolo/yolov11 40 epochs2/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 1120 train, 1120 val
Using 4 dataloader workers
Logging results to yolo/yolov11 40 epochs2
Starting training for 40 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/40      6.98G      1.948      5.512      1.739   
                 Class     Images  Instances      Box(P   

...

      40/40      7.15G     0.5745     0.4452     0.8793   
                 Class     Images  Instances      Box(P   


                   all         19         19      0.949      0.985       0.99      0.819

40 epochs completed in 0.020 hours.
Optimizer stripped from yolo/yolov11 40 epochs2/weights/last.pt, 5.5MB
Optimizer stripped from yolo/yolov11 40 epochs2/weights/best.pt, 5.5MB

Validating yolo/yolov11 40 epochs2/weights/best.pt...
Ultralytics 8.3.94 🚀 Python-3.10.12 torch-2.6.0+cu124 CUDA:0 (Quadro RTX 8000, 48407MiB)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P   


                   all         19         19      0.949      0.984       0.99      0.819
Speed: 0.2ms preprocess, 1.0ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to yolo/yolov11 40 epochs2


lr/pg0,█▇▇▆▆▅▅▄▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr/pg1,▁▂▃▃▄▅▅▆▆▇▇▇████▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
lr/pg2,▁▂▃▃▄▅▅▆▆▇▇▇████▇▇▇▇▆▆▆▅▅▅▅▄▄▄▄▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▁▁▁▁▁▁▁▁▁▁▂▇████▇█████████▇▇▇███████████
metrics/mAP50-95(B),▁▁▁▁▁▁▁▁▁▁▂▆▇▇▆▇▇▇▇████▇▇▇▇▇▇▇█▇████████
metrics/precision(B),▁▁▁▁▁▁▁▁▁▁▃▂█████▇██▇█▇▇▇█▇▇▇▇▇▇▇▇▇█████
metrics/recall(B),▄▁▂▂▃▄▁▁▁▁▁█▄▇▇█▇▇▇▇█▇███▇██████████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▇▄▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁


In [9]:
badset = {'97.dcm', '95.dcm', '178.dcm'}
for i in badset:
    dicom_path = os.path.join(dicom_dir, i)
    img_array = load_dicom(dicom_path)
    img_jpg = dicom_to_jpg(img_array)

    img_filename = i.replace('.dcm', '.jpg')
    img_jpg.save(os.path.join(image_test_dir, img_filename))

In [10]:
results = model(image_test_dir, imgsz=[450,1100], save=True)


WARNING ⚠️ imgsz=[450, 1100] must be multiple of max stride 32, updating to [480, 1120]
image 1/3 /home/abk171/hexai_work/data_yolo/images/test/178.jpg: 480x1120 1 ulnar, 6.8ms
image 2/3 /home/abk171/hexai_work/data_yolo/images/test/95.jpg: 480x1120 (no detections), 7.4ms
image 3/3 /home/abk171/hexai_work/data_yolo/images/test/97.jpg: 480x1120 1 ulnar, 8.0ms
Speed: 3.6ms preprocess, 7.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 1120)
Results saved to yolo/yolov11 40 epochs22
